In [181]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point, mapping
from scipy.ndimage import distance_transform_edt, sobel
from rasterio.features import geometry_mask


import numpy as np 
# Load the shapefile with the polygon
shapefile_path = "crop.shp"
shapefile = gpd.read_file(shapefile_path)

# Load the GeoTIFF
geotiff_path = "36N_50m.tif"
with rasterio.open(geotiff_path) as src:
    # Reproject shapefile to match the GeoTIFF CRS if necessary
    if shapefile.crs != src.crs:
        shapefile = shapefile.to_crs(src.crs)

    # Get the geometry from the shapefile and convert it to GeoJSON format
    geoms = [mapping(geometry) for geometry in shapefile.geometry]

    # Crop the GeoTIFF using the mask function
    dem, out_transform = mask(src, geoms, crop=True)

# Update metadata for the new cropped GeoTIFF
out_meta.update({
    "driver": "GTiff",
    "height": dem.shape[1],
    "width": dem.shape[2],
    "transform": out_transform
})

# path = "cropped_geotiff.tif"
# with rasterio.open(path, "w", **out_meta) as dest:
#     dest.write(dem)

# Side-Scanner Style Hillshade Calculation from Multibeam Data
https://pro.arcgis.com/en/pro-app/latest/tool-reference/3d-analyst/how-hillshade-works.htm
1. Calculate basic Multibeam derivatives:
    - Loading Multibeam geotiff data
    - Compute x and y gradients of the elevation grid
    - Derive slope and aspect grids from gradients
    <br><br>

2. Calculating light source positions:
    - Loading survey line shapefile
    - Find distance and azimuth to the line for every multibeam cell
    - Calculate light zenith based on distance to the line and source depth
    <br><br>
3. Calculate classic Hillshade with improved light source for every cell:
<br>
5. Save the Result

In [193]:
path = "cropped_geotiff.tif"
with rasterio.open(path) as src:
    transform = src.transform
    width = src.width
    height = src.height
    meta = src.meta.copy()
print(width, height)


# Calculate pixel resolution in x and y directions
x_res, y_res = transform[0], -transform[4]

# Calculate gradients in x and y directions
dzdx = sobel(dem, axis=1) / (8 * x_res)
dzdy = sobel(dem, axis=0) / (8 * y_res)

# Calculate slope in degrees
slope = np.arctan(np.sqrt(dzdx**2 + dzdy**2)) 

# Calculate aspect in degrees
aspect = np.arctan2(dzdy, -dzdx)
aspect = np.where(aspect < 0, np.pi*2 + aspect, aspect)

# Update metadata and save slope and aspect grids
meta.update({"dtype": "float32", "count": 1})

with rasterio.open("slope.tif", "w", **meta) as dst:
    dst.write(slope.astype("float32"), [1])

with rasterio.open("aspect.tif", "w", **meta) as dst:
    dst.write(aspect.astype("float32"), [1])
    

2717 166


In [198]:
y_res

50.99807822031765

In [158]:
# Load the polyline shapefile
polyline_path = "surv_line.shp"
polyline = gpd.read_file(polyline_path).geometry.iloc[0]  # Assume only one polyline


#rasterize survey line
line_mask = geometry_mask([mapping(polyline)], transform=transform, invert=True,
                          out_shape=(height, width), all_touched=True).astype(np.uint8)


#calculate distance to survey line
survey_distance, indices = distance_transform_edt(1 - line_mask, return_indices=True)
indices_x, indices_y = indices[1], indices[0]

# Calculate coordinates of each grid cell center and nearest polyline cell
cell_x, cell_y = np.meshgrid(
    np.arange(width) * transform[0] + transform[2] + transform[0] / 2,
    np.arange(height) * transform[4] + transform[5] + transform[4] / 2
)
nearest_x = indices_x * transform[0] + transform[2] + transform[0] / 2
nearest_y = indices_y * transform[4] + transform[5] + transform[4] / 2

# Calculate azimuth in degrees from each cell to its nearest point on the line
dx = nearest_x - cell_x
dy = nearest_y - cell_y
azimuth = np.arctan2(dx, dy) 


path = "azimuth.tif"
with rasterio.open(path, "w", **out_meta) as dest:
    dest.write(np.array([azimuth]))


In [168]:
!pip install terrainpy

ERROR: Could not find a version that satisfies the requirement terrainpy (from versions: none)
ERROR: No matching distribution found for terrainpy


In [ ]:
survey_depth = np.where(dem == -9999,-9999, 200)
zenith = np.tan(survey_distance/survey_depth)

with rasterio.open("zenith.tif", "w", **meta) as dst:
    dst.write(zenith.astype("float32"), [1])

In [155]:


hillshade = 255.0 * ((np.cos(zenith) * np.cos(slope)) +  (np.sin(zenith) * np.sin(slope) * np.cos(azimuth - aspect)))

path = "hillshade.tif"
with rasterio.open(path, "w", **out_meta) as dest:
    dest.write(hillshade)